This notebook demonstrates how to perform distributed training using tf.estimator and tf.data using horovod on Azure Machine learning 

Let's import the required Azure ML Packages...

In [1]:
import azureml.core
import numpy as np
import azureml
from azureml.core import Workspace, Experiment, Run, get_run,Datastore
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.train.dnn import TensorFlow
from azureml.widgets import RunDetails
from azureml.core import Workspace

print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.39


## Initialize workspace

In [10]:
from azureml.core.workspace import Workspace

subscription_id = "fe375bc2-9f1a-4909-ad0d-9319806d5e97"
resource_group = "adb_rg"
workspace_name = "repro"

ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
    
ws.write_config()

# Initialize compute

In [17]:
# Choose a name for your CPU cluster
gpu_cluster_name = "gpucluster"

# Verify that cluster does not exist already
try:
    gpu_cluster = ComputeTarget(workspace=ws, name=gpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6',
                                                           max_nodes=4)
    gpu_cluster = ComputeTarget.create(ws, gpu_cluster_name, compute_config)

gpu_cluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned


# Create experiment

In [18]:
experiment = Experiment(ws, name="tibco_repro")

## Initialize Tensorflow estimator

In [25]:
project_folder = "./"

from azureml.core.runconfig import MpiConfiguration

estimator = TensorFlow(source_directory=project_folder,
                       pip_packages=['numpy'],
                       compute_target=gpu_cluster,
                       entry_script='train.py',
                       node_count=2,
                       distributed_training=MpiConfiguration(),
                       use_gpu=True)

framework_version is not specified, defaulting to version 1.13.


In [26]:
run = experiment.submit(estimator)
print(run)

Run(Experiment: tibco_repro,
Id: tibco_repro_1557948340_aa084838,
Type: azureml.scriptrun,
Status: Queued)


In [27]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…